# Chapter 06: 메모리 (Memory)

이 노트북에서는 에이전트의 단기/장기 메모리 구현을 학습합니다.

## 주요 내용
- 단기 메모리 (Short-term Memory)
- 시맨틱 메모리와 벡터 검색
- LangGraph 체크포인터

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-repo/notebook/ch06_memory.ipynb)


In [ ]:
!pip install -q langchain langchain-openai langgraph python-dotenv


In [ ]:
import os
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except:
    pass
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = "sk-your-api-key-here"


## 1. 메모리 없는 대화 (문제 상황)


In [ ]:
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, MessagesState, START

# LLM 초기화
llm = init_chat_model(model="gpt-5-mini", temperature=0)

def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": response}

# 메모리 없는 그래프 구성
builder = StateGraph(MessagesState)
builder.add_node("call_model", call_model)
builder.add_edge(START, "call_model")
graph_no_memory = builder.compile()

print("⚠️ 메모리 없는 그래프 생성")


In [ ]:
# 메모리 없이 대화 테스트
print("첫 번째 메시지:")
input1 = {"type": "user", "content": "안녕하세요! 제 이름은 민혁입니다."}
for chunk in graph_no_memory.stream({"messages": [input1]}, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

print("\n두 번째 메시지 (이전 대화 기억 못함):")
input2 = {"type": "user", "content": "제 이름이 뭐라고 했죠?"}
for chunk in graph_no_memory.stream({"messages": [input2]}, stream_mode="values"):
    chunk["messages"][-1].pretty_print()


## 2. 메모리 있는 대화 (해결책)

LangGraph의 `MemorySaver`를 사용하여 대화 상태를 유지합니다.


In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# 메모리 체크포인터 설정
memory = MemorySaver()
graph_with_memory = builder.compile(checkpointer=memory)

print("✅ 메모리가 있는 그래프 생성")


In [ ]:
# 메모리와 함께 대화 테스트
config = {"configurable": {"thread_id": "1"}}  # 세션 ID

print("첫 번째 메시지:")
input1 = {"type": "user", "content": "안녕하세요! 제 이름은 민혁입니다."}
for chunk in graph_with_memory.stream({"messages": [input1]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

print("\n두 번째 메시지 (이전 대화 기억함!):")
input2 = {"type": "user", "content": "제 이름이 뭐라고 했죠?"}
for chunk in graph_with_memory.stream({"messages": [input2]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()
